In [ ]:
Ex19_Shoppe&狀態碼為200但無任何資料

In [ ]:
使用偽裝user-agent爬取蝦皮購物網
https://freelancerlife.info/zh/blog/python-web-scraping-user-agent-for-shopee/

In [ ]:
實戰中，雖然只要會使用最基本的requests和beautifulsoup的操作就可以爬許多網站，
但那只是剛好對方的網站伺服器沒有任何檢查措施，就直接將網站的內容回應給你了。

然而只要對方伺服器有任何檢查措施時，你就會得不到你想要的資訊，你可能被拒絕訪問，
又或者是被重新導向到一個新的網頁。為了處理這種狀況，在使用requests套件時，
可以額外加入user agent，使用虛擬的身份，這樣在拜訪網站時，對方的網頁伺服器如果認可你的身份，
你就可以得到正確的網頁內容。

以蝦皮購物作為範例，某些網站需要恰當的使用user-agent，才可以成功的獲取網頁內容。

# 當狀態碼為200，但伺服器卻不回傳給你任何商品

In [ ]:
當你一進入他的首頁時，在搜索欄中輸入你要查找的商品後，你就會跳到商品清單的頁面，請注意這裡的網址。


我以PS4 pro 主機，進行搜索，你可以看到網址變成 https://shopee.tw/search?keyword=PS4%20pro%20%E4%B8%BB%E6%A9%9F 。

很明顯這是以get的方式傳遞你輸入的變數(keyword)給對方的伺服器後台，之後進資料庫比對將最符合的商品傳出來，呈現給你。

因此我們可以試試看手動改一下網址裡的keyword，然後直接用瀏覽器拜訪看看是不是能得到預期的東西。

例如(XBOX one 主機): https://shopee.tw/search?keyword=XBOX%20one%20%E4%B8%BB%E6%A9%9F


沒問題!你在瀏覽器中可以很簡單的直接改網址來獲得不一樣的商品，因此應該可以很簡單的設計一份網路爬蟲程式來抓取你要的商品。

使用我們之前的requests、及beautifulsoup套件，你可以寫出例如以下的程式:

import requests
from bs4 import BeautifulSoup

url = 'https://shopee.tw/search?keyword=PS4%20pro%20%E4%B8%BB%E6%A9%9F'
r = requests.get(url)
print(r.status_code)
print(len(r.text))
Copy
200

似乎沒有問題?狀態碼為200，代表沒有問題?

接著試著使用beautifulesoup去剖析內容，在瀏覽器中檢查元素，可以得知商品是位在
div class=" col-xs-2-4 shopee-search-item-result__item"底下，因此可以使用以下程式嘗試去剖析原始碼:

soup = BeautifulSoup(r.text, 'html.parser')
items = soup.find_all("div", class_="col-xs-2-4 shopee-search-item-result__item")
print(len(items))
Copy
0

沒有任何東西!!

是的，你成功地拜訪了網站(status_code == 200)，但是伺服器卻不回傳給你任何商品。

### 是不是被重新導向了<Response [302]>

In [ ]:
我們可以再檢查看看是不是被重新導向了，被連接到一個新的網址去。

可以使用以下指令檢查:

print(r.history)
print(r.url)
Copy
[<Response [302]>]
https://shopee.tw/search-item/?search=PS4%20pro%20%E4%B8%BB%E6%A9%9F

302 可以簡單的理解為該資源原本確實存在，但已經被臨時改變了位置 。ref: https://zh.wikipedia.org/wiki/HTTP_302

所以導致無法獲得商品資料。網址沒有改變，但是被臨時改變了位置?

為了避免像這樣，status code顯示200，但事實上中途有被重新導向，而誤解為有正確的拜訪網站，
我們可以在request.get中加入allow_redirects=False，

如下:

url = 'https://shopee.tw/search?keyword=PS4%20pro%20%E4%B8%BB%E6%A9%9F'
r = requests.get(url,allow_redirects=False)
print(r.status_code)
Copy
302

直接出現302，現在我們可以避免中途明明被重新導向過，卻顯示一切正常。
但是，我們依然無法獲得商品資訊，該怎麼做呢?

### 使用user-agent，明確的告訴對方伺服器你是誰，然後如果認同你的身分的話就回傳給你正確的結果。

In [ ]:
這就是這篇教學的重點，使用user-agent，明確的告訴對方伺服器你是誰，然後如果認同你的身分的話就回傳給你正確的結果。

我們可以使用fake_useragent隨機產生user-agent，放入requests.get裡面再試一次，

請先安裝fake_useragent ref: https://pypi.org/project/fake-useragent/

pip install fake-useragent
Copy
然後，試試看使用隨機的fake_useragent:

from fake_useragent import UserAgent
ua = UserAgent()
for i in range(5):
    print(ua.random)
    print('*-----------------*')
Copy
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36
*-----------------*
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.124 Safari/537.36
*-----------------*
Mozilla/5.0 (Windows; U; Windows NT 5.1; ru-RU) AppleWebKit/533.19.4 (KHTML, like Gecko) Version/5.0.3 Safari/533.19.4
*-----------------*
Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0
*-----------------*
Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.93 Safari/537.36
*-----------------*

OK!

接著以以下的方式將useragent放入headers在將放入request.get中，重新執行一次。

如果你想當個很有禮貌的爬蟲，可以在額外加入你的email等聯絡方式。

import requests
from fake_useragent import UserAgent

url = 'https://shopee.tw/search?keyword=PS4%20pro%20%E4%B8%BB%E6%A9%9F'
ua = UserAgent()
headers = {
    'User-Agent': ua.random,
    'From': 'YOUR EMAIL ADDRESS'
}

r = requests.get(url,headers=headers,allow_redirects=False)
print(r.status_code)
print(r.history)
print(r.url)

soup = BeautifulSoup(r.text, 'html.parser')
items = soup.find_all("div", class_="col-xs-2-4 shopee-search-item-result__item")
print(len(items))
Copy
200
[]
https://shopee.tw/search-item/?search=PS4%20pro%20%E4%B8%BB%E6%A9%9F
0

這次成功的進入，並沒有被重新導向，但是依然沒有得到商品資訊。

我猜測可能是如果沒有提供user-agent，對方伺服器會將你重新導向，而提供之後，雖然不會重新導向，
但是user-agent並非是伺服器所認可的user-agent，就會拒絕將商品的資訊回傳給你，只丟給你一個空空的頁面。

### 直接看網站的robots.txt來猜測用哪個user-agent

In [ ]:
那麼要用哪個user-agent才有可能會成功呢?我們可以直接看他的robots.txt來進行猜測。

https://shopee.tw/robots.txt

User-Agent:Googlebot
User-Agent:Bingbot
Crawl-delay:0.1
Disallow: /cart/
Disallow: /checkout/
Disallow: /buyer/
Disallow: /user/
Disallow: /me/
Disallow: /order/
Disallow: /daily_discover/
Disallow: /mall/just-for-you/
Disallow: /mall/*-cat.
Disallow: /from_same_shop/
Disallow: /you_may_also_like/
Disallow: *-i.*/similar?from=flash_sale
Disallow: /find_similar_products/
Disallow: /top_products
Disallow: /search*searchPrefill

User-Agent:*
Crawl-delay:1
Disallow: /cart/
Disallow: /checkout/
Disallow: /buyer/
Disallow: /user/
Disallow: /me/
Disallow: /order/
Disallow: /daily_discover/
Disallow: /mall/just-for-you/
Disallow: /mall/*-cat.
Disallow: /from_same_shop/
Disallow: /you_may_also_like/
Disallow: *-i.*/similar
Disallow: /find_similar_products/
Disallow: /top_products
Disallow: /search*searchPrefill
Copy
既然robots.txt中第一個提到的user-agent就是Googlebot，何不以Googlebot來試試呢?

url = 'https://shopee.tw/search?keyword=PS4%20pro%20%E4%B8%BB%E6%A9%9F'
headers = {
    'User-Agent': 'Googlebot',
    'From': 'YOUR EMAIL ADDRESS'
}

r = requests.get(url,headers=headers,allow_redirects=True)
print(r.status_code)
print(r.history)
print(r.url)

soup = BeautifulSoup(r.text, 'html.parser')
items = soup.find_all("div", class_="col-xs-2-4 shopee-search-item-result__item")
print(len(items))
Copy
200
[]
https://shopee.tw/search?keyword=PS4%20pro%20%E4%B8%BB%E6%A9%9F
50

成功!! 有50個商品!!

使用了Googlebot，對方伺服器認可了你所偽裝的身分，正確的將商品資訊傳給了你。

In [ ]:
這次我們試圖以requests拜訪蝦皮購物網，經過多方嘗試，終於在設定user-agent等於Googlebot的情況下，成功地獲得了商品資訊。
筆者也使用過robots.txt裡提到的Bingbot，但是也無法獲得商品資訊，只能說是湊巧成功了。
是不是非常驚險刺激呢?爬蟲程式最有趣的地方就在於猜測對方伺服器後台是如何傳送資料，然後設法以各種手段獲得內容。

另外，仔細檢查蝦皮購物網的內容後，事實上對方是有api可以快速呼叫商品內容的，
如: https://shopee.tw/api/v2/search_items/?by=relevancy&keyword=PS4%20pro%20%E4%B8%BB%E6%A9%9F&limit=20&newest=0&order=desc&page_type=search
，會得到包含商品資訊的json檔，然而一樣在python底下使用requests試圖爬取時，要用Googlebot才有辦法獲得該資訊。

當我們能夠成功地爬取購物網的商品內容後，就有很多可能的應用，例如:

(1) 編寫多個不同電商網站的爬蟲程式，放在一起，變成一個比價程式，讓使用者只要輸入商品名稱，
就能跳出各大電商網站該商品的價格，讓使用者可以找到哪個購物網有比較便宜的價格。

(2) 讓使用者能夠快速的搜尋多項商品並且比較，例如，可以快速地比較PS4和XBox的價格大約為在哪裡。

(3) 可以將電商網站的爬蟲程式寫入排程器裡面(如linux底下的crontab)每隔一段時間執行一次，
將商品價格儲存下來，如此，經過一段時間，我們就可以進行商品價格波動的分析。

### 本章節程式碼

In [ ]:
import requests
from bs4 import BeautifulSoup

def shopee_scraper(keyword,n_page=0,used=False,new=False):
    '''
    參數說明:
        keyword: 商品名稱關鍵字
        n_page: 第幾頁(每頁有50個商品)
        used: 是否為二手商品?
        new: 是否為新商品?
    '''
    url = f'https://shopee.tw/search?keyword={keyword}&page={n_page}&sortBy=relevancy'
    if used:
        url += '&newItem=true'
    if new:
        url += '&usedItem=true'
    
    headers = {
        'User-Agent': 'Googlebot',
        'From': 'YOUR EMAIL ADDRESS'
    }
    
    r = requests.get(url,headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    contents = soup.find_all("div", class_="_1NoI8_ _2gr36I")
    prices = soup.find_all("span", class_="_341bF0")
    all_items = soup.find_all("div", class_="col-xs-2-4 shopee-search-item-result__item")
    links = [i.find('a').get('href') for i in all_items]
    
    for c, p, l in zip(contents, prices, links):
        print(c.contents[0])
        print(p.contents[0])
        print('https://shopee.tw/'+l)
        print('*---------------------------------*')

#### 各國蝦皮網頁格式都一樣，以下Code只要改一下網址就可以爬不同國家

# 爬取【菲律賓蝦皮】搜尋頁面簡易商品資料_20200610完整版
1. 使用針對蝦皮的headers = {"user-agent": "Googlebot"}取得解析網頁資料
2. 先取得搜尋頁數的網址List
3. 直接爬取網址List之商品資料

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fake_useragent import UserAgent


def get_urls(url, query, start_page, end_page): 
    urls = []
    for page in range(start_page, end_page+1):
        urls.append(url.format(query, page-1))    #query帶入url的{0}、page帶入{1}
    return urls

def get_resource(url):
    ua = UserAgent()
    headers = {"user-agent": "Googlebot"}
    return requests.get(url, headers=headers)

def parse_html(html_str):
    return BeautifulSoup(html_str, "lxml")

def get_goods(soup):
    goods = []
    rows = soup.find_all("div",class_="col-xs-2-4 shopee-search-item-result__item")

    for row in rows:

        try:
            name = row.find("div", class_="_1NoI8_ _16BAGk").text
        except:
            name = None

        try:
            price = row.find("span", class_="_341bF0").text
        except:
            price = None
            
        try:
            Original_price = row.find("div", class_="_1w9jLI QbH7Ig U90Nhh").text
        except:
            Original_price = None

        try:
            sold = row.find("div", class_="_18SLBt").text
        except:
            sold = None

        try:
            link = "https://ph.xiapibuy.com/" + row.find("a").get('href')
        except:
            link = None
        
        good= [name, price, Original_price, sold, link]
        goods.append(good)
    return goods

def web_scraping_bot(urls):
    all_goods = [["品名","價格(單位:千)","原價(單位:千)","售出量", "網址"]]  #巢狀清單
    page = 1
    
    for url in urls:
        print("抓取: 第" + str(page) + "頁 網路資料中...")
        page = page + 1
        r = get_resource(url)
        if r.status_code == requests.codes.ok:
            soup = parse_html(r.text)
            goods = get_goods(soup)
            all_goods = all_goods + goods
            print("等待5秒鐘...")
            
            #當目前頁數=所有頁數時，跳出迴圈
            now_page = soup.find("span", class_="shopee-mini-page-controller__current").text
            all_page = soup.find("span", class_="shopee-mini-page-controller__total").text
            if now_page == all_page:
                break   #已經沒有下一頁
            time.sleep(5) 
        else:
            print("HTTP請求錯誤...")

    return all_goods

def save_to_csv(items, file):
    with open(file, "w+", newline="", encoding="utf_8_sig") as fp:  #utf_8_sig:能讓輸出的csv正確顯示中文(utf_8會有亂碼)
        writer = csv.writer(fp)
        for item in items:
            writer.writerow(item)

if __name__ == "__main__":
    # 目標URL網址
    """直接在蝦皮搜尋"""
    #url = "https://ph.xiapibuy.com/search?keyword={0}&page={1}"
    """在電腦配件中搜尋"""
    url = "https://ph.xiapibuy.com/search?category=18596&keyword={0}&subcategory=18613&page={1}"
    """在電腦遊戲中搜尋"""
    #url = "https://ph.xiapibuy.com/search?category=20718&keyword={0}&subcategory=20729&page={1}"

    print(get_resource(url).status_code)

    urls = get_urls(url, "genius", 1, 3)
    print(urls)

    goods = web_scraping_bot(urls)
    df = pd.DataFrame(goods)       #用dataframe列出
    print(df)
    
    save_to_csv(goods, "x.csv")

# 爬取【印尼蝦皮】搜尋頁面簡易商品資料_20200610完整版
1. 使用針對蝦皮的headers = {"user-agent": "Googlebot"}取得解析網頁資料
2. 先取得搜尋頁數的網址List
3. 直接爬取網址List之商品資料

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fake_useragent import UserAgent


def get_urls(url, query, start_page, end_page): 
    urls = []
    for page in range(start_page, end_page+1):
        urls.append(url.format(query, page-1))    #query帶入url的{0}、page帶入{1}
    return urls

def get_resource(url):
    ua = UserAgent()
    headers = {"user-agent": "Googlebot"}
    return requests.get(url, headers=headers)

def parse_html(html_str):
    return BeautifulSoup(html_str, "lxml")

def get_goods(soup):
    goods = []
    rows = soup.find_all("div",class_="col-xs-2-4 shopee-search-item-result__item")

    for row in rows:

        try:
            name = row.find("div", class_="_1NoI8_ _16BAGk").text
        except:
            name = None

        try:
            price = row.find("span", class_="_341bF0").text
        except:
            price = None
            
        try:
            Original_price = row.find("div", class_="_1w9jLI QbH7Ig U90Nhh").text.replace("Rp","")
        except:
            Original_price = None

        try:
            sold = row.find("div", class_="_18SLBt").text.replace(" Terjual","")
            #假如字串有包含RB，將RB取代掉，並將字串轉為浮點數，再乘以1000
            if "RB" in sold:
                sold = sold.replace("RB","")
                sold = float(sold)*1000
        except:
            sold = None

        try:
            link = "https://id.xiapibuy.com/" + row.find("a").get('href')
        except:
            link = None
        
        good= [name, price, Original_price, sold, link]
        goods.append(good)
    return goods

def web_scraping_bot(urls):
    all_goods = [["品名","價格(單位:千)","原價(單位:千)","售出量", "網址"]]  #巢狀清單
    page = 1
    
    for url in urls:
        print("抓取: 第" + str(page) + "頁 網路資料中...")
        page = page + 1
        r = get_resource(url)
        if r.status_code == requests.codes.ok:
            soup = parse_html(r.text)
            goods = get_goods(soup)
            all_goods = all_goods + goods
            print("等待5秒鐘...")
            
            #當目前頁數=所有頁數時，跳出迴圈
            now_page = soup.find("span", class_="shopee-mini-page-controller__current").text
            all_page = soup.find("span", class_="shopee-mini-page-controller__total").text
            if now_page == all_page:
                break   #已經沒有下一頁
            time.sleep(5) 
        else:
            print("HTTP請求錯誤...")

    return all_goods

def save_to_csv(items, file):
    with open(file, "w+", newline="", encoding="utf_8_sig") as fp:  #utf_8_sig:能讓輸出的csv正確顯示中文(utf_8會有亂碼)
        writer = csv.writer(fp)
        for item in items:
            writer.writerow(item)

if __name__ == "__main__":
    # 目標URL網址
    """直接在蝦皮搜尋"""
    #url = "https://id.xiapibuy.com/search?keyword={0}&page={1}"
    """在電腦配件中搜尋"""
    #url = "https://id.xiapibuy.com/search?category=134&keyword={0}&page={1}"
    """直接搜尋品牌"""
    url = "https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal={0}&page={1}"

    print(get_resource(url).status_code)

    urls = get_urls(url, "genius", 1, 3)
    print(urls)

    goods = web_scraping_bot(urls)
    df = pd.DataFrame(goods)       #用dataframe列出
    print(df)
    
    save_to_csv(goods, "x.csv")

# 遍歷【印尼蝦皮】各商品爬取細節_20200609完整版
1. 使用針對蝦皮的headers = {"user-agent": "Googlebot"}取得解析網頁資料
2. 先取得搜尋頁數的網址List
3. 進入每個商品頁面
4. 爬取進入頁面之商品細節

In [3]:
import re, time, requests, csv
from bs4 import BeautifulSoup
import csv

#解析(蝦皮headers要用Googlebot)
def get_soup(url):
    headers = {"user-agent": "Googlebot"}
    res = requests.get(url, headers=headers)
    return BeautifulSoup(res.text, 'lxml')

#搜尋網址&換頁
def get_urls(url, query, start_page, end_page): 
    urls = []
    
    for page in range(start_page, end_page+1):
        urls.append(url.format(query, page))    #query帶入url的{0}、page帶入{1}
    print(urls)    
    return urls

# 依序爬取每頁點入網址
def FindLinks(pages):
    linklist = []
    for page in pages:  
        soup = get_soup(page)
        links = soup.find_all("div",class_="col-xs-2-4 shopee-search-item-result__item")
        for link in links:
            k = "https://id.xiapibuy.com/" + link.find("a").get('href')
            linklist.append(k)
    return linklist

# 爬取點入分頁資料
def get_goods(url):
    goods = []
    rows = get_soup(url)

    for row in rows:

        try:
            name = row.find('div', class_="qaNIZv").text
        except:
            name = None

        try:
            #price = row.find('div', class_="_3n5NQx").text.replace("Rp","")
            price = row.select_one("._3n5NQx").get_text().replace("Rp","")
        except:
            price = None
            
        try:
            Original_price = row.find("div", class_="_3_ISdg").text.replace("Rp","")
        except:
            Original_price = None
            
        try:
            star = row.find('div', '_3Oj5_n _2z6cUg').text
        except:
            star = None

        try:
            #reviews = row.select("._3Oj5_n")[1].get_text()
            reviews = row.find_all('div', '_3Oj5_n')[1].text
        except:
            reviews = None
            
        try:
            sold = row.find('div', '_22sp0A').text
        except:
            sold = None
            
        try:
            #stock = row.select("._1FzU2Y .items-center div+ div")[0].get_text()
            stock = row.find("div", "flex items-center _1FzU2Y").get_text()
        except:
            stock = None
        
        try:
            seller = row.select_one("._3Lybjn").get_text()
        except:
            seller = None
            
        try:
            seller_link = "https://id.xiapibuy.com" + row.find("a", "btn btn-light btn--s btn--inline btn-light--link Ed2lAD").get('href')
        except:
            seller_link = None
            
        try:
            seller_from = row.select(".kIo6pj")[-1].div.get_text()
            #seller_from = row.find_all("kIo6pj")[-1].text
        except:
            seller_from = None
            
        try:
            category = row.select_one(".kIo6pj").get_text().replace("Kategori", "")
        except:
            category = None

        try:
            brand = row.select_one("._2H-513").get_text()
        except:
            brand = None

        try:
            description = row.find("div", "_2u0jt9").get_text()
        except:
            description = None
            
        try:
            URL = url
        except:
            URL = None
            
        good= [name, price, Original_price, star, reviews, sold, stock, seller, seller_link, seller_from, category, brand, description, URL]
        goods.append(good)
        
    return goods[1]  #因為不知為何第[0]列都會出現一排None，只好取第[1]列

# 將每一個點入頁面的List依序爬取
def scraping(urls):
    all_goods = [["name","price","Original_price","star","reviews","sold","stock","seller","seller_link","seller_from","category","brand","description", "URL"]]
    for idx,i in enumerate(FindLinks(urls)):  #記錄目前進行的迴圈次數，配上總迴圈次數，可做為進度條使用。
        print("Crawing No." + str(idx+1) + " Item in Total:" + str(len(FindLinks(urls))) + "Item")
        
        goods = get_goods(i)
        time.sleep(0.2)
        all_goods.append(goods)
    return all_goods

#存成CSV
def save_to_csv(items, file):
    with open(file, "w+", newline="", encoding="utf_8_sig") as fp:  #utf_8_sig:能讓輸出的csv正確顯示中文(utf_8會有亂碼)
        writer = csv.writer(fp)
        for item in items:
            writer.writerow(item)
    
# 開始爬蟲
if __name__ == "__main__":
    """直接在蝦皮搜尋"""
    #url = "https://id.xiapibuy.com/search?keyword={0}&page={1}"
    """在電腦配件中搜尋"""
    #url = "https://id.xiapibuy.com/search?category=134&keyword={0}&page={1}"
    """直接搜尋品牌"""
    #url = "https://id.xiapibuy.com/search?attrId=14478&attrName=Merek&attrVal={0}&page={1}"
    """在電腦配件中搜尋+照暢銷排序"""
    url = "https://id.xiapibuy.com/search?category=134&keyword={0}&page={1}&sortBy=sales"
    
    urls = get_urls(url, "genius", 0, 50)
    
    m = scraping(urls)
    save_to_csv(m, "m.csv")

['https://id.xiapibuy.com/search?category=134&keyword=genius&page=0&sortBy=sales', 'https://id.xiapibuy.com/search?category=134&keyword=genius&page=1&sortBy=sales', 'https://id.xiapibuy.com/search?category=134&keyword=genius&page=2&sortBy=sales', 'https://id.xiapibuy.com/search?category=134&keyword=genius&page=3&sortBy=sales', 'https://id.xiapibuy.com/search?category=134&keyword=genius&page=4&sortBy=sales', 'https://id.xiapibuy.com/search?category=134&keyword=genius&page=5&sortBy=sales', 'https://id.xiapibuy.com/search?category=134&keyword=genius&page=6&sortBy=sales', 'https://id.xiapibuy.com/search?category=134&keyword=genius&page=7&sortBy=sales', 'https://id.xiapibuy.com/search?category=134&keyword=genius&page=8&sortBy=sales', 'https://id.xiapibuy.com/search?category=134&keyword=genius&page=9&sortBy=sales', 'https://id.xiapibuy.com/search?category=134&keyword=genius&page=10&sortBy=sales', 'https://id.xiapibuy.com/search?category=134&keyword=genius&page=11&sortBy=sales', 'https://id.x

KeyboardInterrupt: 

# 遍歷【台灣蝦皮】各商品爬取細節_20200610完整版
1. 先取得搜尋頁數的網址List
2. 進入每個商品頁面
3. 爬取進入頁面之商品細節

In [ ]:
import re, time, requests, csv
from bs4 import BeautifulSoup
import csv

#解析(蝦皮headers要用Googlebot)
def get_soup(url):
    headers = {"user-agent": "Googlebot"}
    res = requests.get(url, headers=headers)
    return BeautifulSoup(res.text, 'lxml')

#搜尋網址&換頁
def get_urls(url, query, start_page, end_page): 
    urls = []
    
    for page in range(start_page, end_page+1):
        urls.append(url.format(query, page))    #query帶入url的{0}、page帶入{1}
    print(urls)    
    return urls

# 依序爬取每頁點入網址
def FindLinks(pages):
    linklist = []
    for page in pages:  
        soup = get_soup(page)
        links = soup.find_all("div",class_="col-xs-2-4 shopee-search-item-result__item")
        for link in links:
            k = "https://shopee.tw/" + link.find("a").get('href')
            linklist.append(k)
    return linklist

# 爬取點入分頁資料
def get_goods(url):
    goods = []
    rows = get_soup(url)

    for row in rows:

        try:
            name = row.find('div', class_="qaNIZv").text
        except:
            name = None

        try:
            #price = row.find('div', class_="_3n5NQx").text
            price = row.select_one("._3n5NQx").get_text()
        except:
            price = None
            
        try:
            Original_price = row.find("div", class_="_3_ISdg").text
        except:
            Original_price = None
            
        try:
            star = row.find('div', '_3Oj5_n _2z6cUg').text
        except:
            star = None

        try:
            #reviews = row.select("._3Oj5_n")[1].get_text()
            reviews = row.find_all('div', '_3Oj5_n')[1].text
        except:
            reviews = None
            
        try:
            sold = row.find('div', '_22sp0A').text
        except:
            sold = None
            
        try:
            #stock = row.select("._1FzU2Y .items-center div+ div")[0].get_text()
            stock = row.find("div", "flex items-center _1FzU2Y").get_text()
        except:
            stock = None
        
        try:
            seller = row.select_one("._3Lybjn").get_text()
        except:
            seller = None
            
        try:
            seller_link = "https://shopee.tw" + row.find("a", "btn btn-light btn--s btn--inline btn-light--link Ed2lAD").get('href')
        except:
            seller_link = None
            
        try:
            seller_from = row.select(".kIo6pj")[-1].div.get_text()
            #seller_from = row.find_all("kIo6pj")[-1].text
        except:
            seller_from = None
            
        try:
            category = row.select_one(".kIo6pj").get_text().replace("Kategori", "")
        except:
            category = None

        try:
            brand = row.select_one("._2H-513").get_text()
        except:
            brand = None

        try:
            description = row.find("div", "_2u0jt9").get_text()
        except:
            description = None
            
        try:
            URL = url
        except:
            URL = None
            
        good= [name, price, Original_price, star, reviews, sold, stock, seller, seller_link, seller_from, category, brand, description, URL]
        goods.append(good)
        
    return goods[1]  #因為不知為何第[0]列都會出現一排None，只好取第[1]列

# 將每一個點入頁面的List依序爬取
def scraping(urls):
    all_goods = [["name","price","Original_price","star","reviews","sold","stock","seller","seller_link","seller_from","category","brand","description", "URL"]]
    for idx,i in enumerate(FindLinks(urls)):  #記錄目前進行的迴圈次數，配上總迴圈次數，可做為進度條使用。
        print("Crawing No." + str(idx+1) + " Item in Total:" + str(len(FindLinks(urls))) + "Item")
        
        goods = get_goods(i)
        time.sleep(0.2)
        all_goods.append(goods)
    return all_goods

#存成CSV
def save_to_csv(items, file):
    with open(file, "w+", newline="", encoding="utf_8_sig") as fp:  #utf_8_sig:能讓輸出的csv正確顯示中文(utf_8會有亂碼)
        writer = csv.writer(fp)
        for item in items:
            writer.writerow(item)
    
# 開始爬蟲
if __name__ == "__main__":
    """直接在蝦皮搜尋"""
    url = "https://shopee.tw/search?keyword={0}&page={1}"
    """在電腦配件中搜尋"""
    #url = "https://shopee.tw/search?category=134&keyword={0}&page={1}"
    """直接搜尋品牌"""
    #url = "https://shopee.tw/search?attrId=14478&attrName=Merek&attrVal={0}&page={1}"
    
    urls = get_urls(url, "昆盈", 0, 1)
    
    m = scraping(urls)
    save_to_csv(m, "m.csv")

In [ ]:
菲律賓

In [ ]:
import re, time, requests, csv
from bs4 import BeautifulSoup
import csv

#解析(蝦皮headers要用Googlebot)
def get_soup(url):
    headers = {"user-agent": "Googlebot"}
    res = requests.get(url, headers=headers)
    return BeautifulSoup(res.text, 'lxml')

#搜尋網址&換頁
def get_urls(url, query, start_page, end_page): 
    urls = []
    
    for page in range(start_page, end_page+1):
        urls.append(url.format(query, page))    #query帶入url的{0}、page帶入{1}
    print(urls)    
    return urls

# 依序爬取每頁點入網址
def FindLinks(pages):
    linklist = []
    for page in pages:  
        soup = get_soup(page)
        links = soup.find_all("div",class_="col-xs-2-4 shopee-search-item-result__item")
        for link in links:
            k = "https://ph.xiapibuy.com/" + link.find("a").get('href')
            linklist.append(k)
    return linklist

# 爬取點入分頁資料
def get_goods(url):
    goods = []
    rows = get_soup(url)

    for row in rows:

        try:
            name = row.find('div', class_="qaNIZv").text
        except:
            name = None

        try:
            #price = row.find('div', class_="_3n5NQx").text.replace("Rp","")
            price = row.select_one("._3n5NQx").get_text().replace("Rp","")
        except:
            price = None
            
        try:
            Original_price = row.find("div", class_="_3_ISdg").text.replace("Rp","")
        except:
            Original_price = None
            
        try:
            star = row.find('div', '_3Oj5_n _2z6cUg').text
        except:
            star = None

        try:
            #reviews = row.select("._3Oj5_n")[1].get_text()
            reviews = row.find_all('div', '_3Oj5_n')[1].text
        except:
            reviews = None
            
        try:
            sold = row.find('div', '_22sp0A').text
        except:
            sold = None
            
        try:
            #stock = row.select("._1FzU2Y .items-center div+ div")[0].get_text()
            stock = row.find("div", "flex items-center _1FzU2Y").get_text()
        except:
            stock = None
        
        try:
            seller = row.select_one("._3Lybjn").get_text()
        except:
            seller = None
            
        try:
            seller_link = "https://ph.xiapibuy.com" + row.find("a", "btn btn-light btn--s btn--inline btn-light--link Ed2lAD").get('href')
        except:
            seller_link = None
            
        try:
            seller_from = row.select(".kIo6pj")[-1].div.get_text()
            #seller_from = row.find_all("kIo6pj")[-1].text
        except:
            seller_from = None
            
        try:
            category = row.select_one(".kIo6pj").get_text().replace("Kategori", "")
        except:
            category = None

        try:
            brand = row.select_one("._2H-513").get_text()
        except:
            brand = None

        try:
            description = row.find("div", "_2u0jt9").get_text()
        except:
            description = None
            
        try:
            URL = url
        except:
            URL = None
            
        good= [name, price, Original_price, star, reviews, sold, stock, seller, seller_link, seller_from, category, brand, description, URL]
        goods.append(good)
        
    return goods[1]  #因為不知為何第[0]列都會出現一排None，只好取第[1]列

# 將每一個點入頁面的List依序爬取
def scraping(urls):
    all_goods = [["name","price","Original_price","star","reviews","sold","stock","seller","seller_link","seller_from","category","brand","description", "URL"]]
    for idx,i in enumerate(FindLinks(urls)):  #記錄目前進行的迴圈次數，配上總迴圈次數，可做為進度條使用。
        print("Crawing No." + str(idx+1) + " Item in Total:" + str(len(FindLinks(urls))) + "Item")
        
        goods = get_goods(i)
        time.sleep(0.2)
        all_goods.append(goods)
    return all_goods

#存成CSV
def save_to_csv(items, file):
    with open(file, "w+", newline="", encoding="utf_8_sig") as fp:  #utf_8_sig:能讓輸出的csv正確顯示中文(utf_8會有亂碼)
        writer = csv.writer(fp)
        for item in items:
            writer.writerow(item)
    
# 開始爬蟲
if __name__ == "__main__":
    #在電腦週邊搜尋
    url = "https://ph.xiapibuy.com/search?category=18596&keyword={0}&subcategory=18613&page={1}"
    
    urls = get_urls(url, "genius", 0, 3)
    
    m = scraping(urls)
    save_to_csv(m, "m.csv")

# 遍歷【越南蝦皮】各商品爬取細節_20200619完整版
1. 先取得搜尋頁數的網址List
2. 進入每個商品頁面
3. 爬取進入頁面之商品細節

In [1]:
import re, time, requests, csv
from bs4 import BeautifulSoup
import csv

#解析(蝦皮headers要用Googlebot)
def get_soup(url):
    headers = {"user-agent": "Googlebot"}
    res = requests.get(url, headers=headers)
    return BeautifulSoup(res.text, 'lxml')

#搜尋網址&換頁
def get_urls(url, query, start_page, end_page): 
    urls = []
    
    for page in range(start_page, end_page+1):
        urls.append(url.format(query, page))    #query帶入url的{0}、page帶入{1}
    print(urls)    
    return urls

# 依序爬取每頁點入網址
def FindLinks(pages):
    linklist = []
    for page in pages:  
        soup = get_soup(page)
        links = soup.find_all("div",class_="col-xs-2-4 shopee-search-item-result__item")
        for link in links:
            k = "https://vn.xiapibuy.com/" + link.find("a").get('href')
            linklist.append(k)
    return linklist

# 爬取點入分頁資料
def get_goods(url):
    goods = []
    rows = get_soup(url)

    for row in rows:

        try:
            name = row.find('div', class_="qaNIZv").text
        except:
            name = None

        try:
            #price = row.find('div', class_="_3n5NQx").text.replace("Rp","")
            price = row.select_one("._3n5NQx").get_text().replace("Rp","")
        except:
            price = None
            
        try:
            Original_price = row.find("div", class_="_3_ISdg").text.replace("Rp","")
        except:
            Original_price = None
            
        try:
            star = row.find('div', '_3Oj5_n _2z6cUg').text
        except:
            star = None

        try:
            #reviews = row.select("._3Oj5_n")[1].get_text()
            reviews = row.find_all('div', '_3Oj5_n')[1].text
        except:
            reviews = None
            
        try:
            sold = row.find('div', '_22sp0A').text
        except:
            sold = None
            
        try:
            #stock = row.select("._1FzU2Y .items-center div+ div")[0].get_text()
            stock = row.find("div", "flex items-center _1FzU2Y").get_text()
        except:
            stock = None
        
        try:
            seller = row.select_one("._3Lybjn").get_text()
        except:
            seller = None
            
        try:
            seller_link = "https://vn.xiapibuy.com" + row.find("a", "btn btn-light btn--s btn--inline btn-light--link Ed2lAD").get('href')
        except:
            seller_link = None
            
        try:
            seller_from = row.select(".kIo6pj")[-1].div.get_text()
            #seller_from = row.find_all("kIo6pj")[-1].text
        except:
            seller_from = None
            
        try:
            category = row.select_one(".kIo6pj").get_text().replace("Kategori", "")
        except:
            category = None

        try:
            brand = row.select_one("._2H-513").get_text()
        except:
            brand = None

        try:
            description = row.find("div", "_2u0jt9").get_text()
        except:
            description = None
            
        try:
            URL = url
        except:
            URL = None
            
        good= [name, price, Original_price, star, reviews, sold, stock, seller, seller_link, seller_from, category, brand, description, URL]
        goods.append(good)
        
    return goods[1]  #因為不知為何第[0]列都會出現一排None，只好取第[1]列

# 將每一個點入頁面的List依序爬取
def scraping(urls):
    all_goods = [["name","price","Original_price","star","reviews","sold","stock","seller","seller_link","seller_from","category","brand","description", "URL"]]
    for idx,i in enumerate(FindLinks(urls)):  #記錄目前進行的迴圈次數，配上總迴圈次數，可做為進度條使用。
        print("Crawing No." + str(idx+1) + " Item in Total:" + str(len(FindLinks(urls))) + "Item")
        
        goods = get_goods(i)
        time.sleep(0.2)
        all_goods.append(goods)
    return all_goods

#存成CSV
def save_to_csv(items, file):
    with open(file, "w+", newline="", encoding="utf_8_sig") as fp:  #utf_8_sig:能讓輸出的csv正確顯示中文(utf_8會有亂碼)
        writer = csv.writer(fp)
        for item in items:
            writer.writerow(item)
    
# 開始爬蟲
if __name__ == "__main__":
    #在電腦週邊搜尋+銷售中
    url = "https://vn.xiapibuy.com/search?category=13030&keyword={0}&page={1}&sortBy=sales"
    urls = get_urls(url, "genius", 11, 11)
    
    m = scraping(urls)
    save_to_csv(m, "m.csv")

['https://vn.xiapibuy.com/search?category=13030&keyword=genius&page11&sortBy=sales']
Crawing No.1 Item in Total:50Item
Crawing No.2 Item in Total:50Item
Crawing No.3 Item in Total:50Item
Crawing No.4 Item in Total:50Item
Crawing No.5 Item in Total:50Item
Crawing No.6 Item in Total:50Item
Crawing No.7 Item in Total:50Item
Crawing No.8 Item in Total:50Item
Crawing No.9 Item in Total:50Item
Crawing No.10 Item in Total:50Item
Crawing No.11 Item in Total:50Item
Crawing No.12 Item in Total:50Item
Crawing No.13 Item in Total:50Item
Crawing No.14 Item in Total:50Item
Crawing No.15 Item in Total:50Item
Crawing No.16 Item in Total:50Item
Crawing No.17 Item in Total:50Item
Crawing No.18 Item in Total:50Item
Crawing No.19 Item in Total:50Item
Crawing No.20 Item in Total:50Item
Crawing No.21 Item in Total:50Item
Crawing No.22 Item in Total:50Item
Crawing No.23 Item in Total:50Item
Crawing No.24 Item in Total:50Item
Crawing No.25 Item in Total:50Item
Crawing No.26 Item in Total:50Item
Crawing No.27 

In [ ]:
# 撈取深網中的資料-蝦皮購物API
https://freelancerlife.info/zh/blog/%E6%92%88%E5%8F%96%E6%B7%B1%E7%B6%B2%E4%B8%AD%E7%9A%84%E8%B3%87%E6%96%99-%E8%9D%A6%E7%9A%AE%E8%B3%BC%E7%89%A9api/

In [ ]:
深層網路(Deep Web)是指無法被搜索引擎搜索到的網站內容。使用瀏覽器拜訪一個網站，除了一個主要的html檔之外，對方的伺服器同時會載入許多檔案讓這個網頁能正常運作，例如:圖片檔、css、javascript、json檔等，這些檔案也都會有各自的網址，但並不會被搜索引擎給包含到，也就屬於深層網路的部分。

當你要進行爬蟲時，務必要先檢查這些深層網路的檔案內容，有時候你想要的網頁資料通通都藏在這裡，讓你可以很簡便的抓取。

要如何查看這些檔案呢?很簡單，使用你的瀏覽器就可以了。

In [ ]:
首先前往要檢查的網站，在任意地方按下右鍵，選擇檢查網站，此時你的瀏覽器視窗會被新冒出來的檢查視窗分割。

點選網路，按下重新整理(或F5)，這時候就可以一一點選查看所右網頁讀取進來的檔案。

延續上一篇教學，我依然使用蝦皮購物網進行示範，操作流程示範影片如下:

In [ ]:
要檢查什麼呢?

首先查看html這裡，這個主網頁是哪種請求方法?

In [ ]:
這個網站的請求方法是GET。除了GET之外還會有POST的請求方法，這就必須使用不同的程式碼來進行爬蟲。
而如果是POST的話，則需要再察看參數，在使用進行爬蟲時需要在額外輸入這些參數，這部分留到下一篇教學再仔細跟各位介紹。

![Image of Yaktocat](https://freelancerlife.info/media/original_images/check_parameter.PNG)

In [ ]:
這裡顯示參數的名稱為keyword，內容為switch主機，因為是以GET的方式請求，因此參數就直接在網址裡面。

接著檢查XHR，有些網站會以額外的方式將資料以json或xml的格式送進網頁中，如果你能直接獲取這些檔案，那就不用去處理解析網站原始碼的部分。

在XHR中有許多網址，逐一檢查，這裡很多網址都包含了api，很有可能就是用來傳輸資料給網頁。

例如這裡的網址就非常的可疑， 
https://shopee.tw/api/v2/search_items/?by=relevancy&keyword=switch%E4%B8%BB%E6%A9%9F&limit=50&newest=0&order=desc&page_type=search

，既提到api又寫到seach_items，很有可能資料都在這裡喔。

In [ ]:
直接用瀏覽器前往這個網址，商品資料果然在這裡。
但是很多資料都是缺值，例如我們最在乎的商品價格。所以無法以api的方式查詢商品價格嗎?

我們在嘗試隨便前往任一商品的分頁檢查，看看有沒有類似的api來傳送商品價格。

https://shopee.tw/Nintendo-Switch%E4%B8%BB%E6%A9%9F-%E9%9B%BB%E5%85%89%E7%B4%85%E8%97%8D(%E5%8F%B0%E7%81%A3%E5%85%AC%E5%8F%B8%E8%B2%A8)-i.39500389.2495200573

In [ ]:
找到一個很可疑的網址，這裡面會不會有商品價格呢?

https://shopee.tw/api/v2/item/get?itemid=2495200573&shopid=39500389

In [ ]:
有!

看來我們可以利用這兩個api來進行爬蟲，抓取商品價格。

邏輯大概是這樣子的，在第一個api中輸入關鍵字，將裡面的itemid和shopid抓取出來，放到第二個api底下去查詢商品價格。

OK~我們可以開始寫程式以這條路徑爬取商品價格。

In [ ]:
#使用requests與json套件，json套件可以將json格式轉成python的dict，讓我們可以很簡單的讀取到內容。
import requests
import json

In [ ]:
嘗試獲取第一個api的資訊，商品名稱、itemid、shopid:

一樣要使用User-Agent為Googlebot才可以喔，不然會被重新導向，抓不到任何資料。

In [10]:
url = 'https://shopee.tw/api/v2/search_items/?by=relevancy&keyword=switch&limit=50'

headers = {
        'User-Agent': 'Googlebot',
    }

r = requests.get(url,headers=headers)

api1_data = json.loads(r.text)

# 查看第8筆資料的內容:
print(api1_data['items'][8]['name'])
print(api1_data['items'][8]['itemid'])
print(api1_data['items'][8]['shopid'])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
好，接著使用這組itemid和shopid來爬取這筆商品的價格。

In [ ]:
itemid = api1_data['items'][8]['itemid']
shopid = api1_data['items'][8]['shopid']
url2 = f'https://shopee.tw/api/v2/item/get?itemid={itemid}&shopid={shopid}'

r = requests.get(url2,headers=headers)
api2_data = json.loads(r.text)
print(api2_data['item']['price'])

In [ ]:
多出了五個零，沒關係，移掉就好。

OK~

我們可以很簡單的編寫爬蟲程式獲取商品價格，最後在把上面的程式改寫成好用函式吧~

In [ ]:
import time
import requests
import json

def shopeeAPI_Scraper(keyword, n_items):

    url1 = f'https://shopee.tw/api/v2/search_items/?by=relevancy&keyword={keyword}&limit={n_items}'
    headers = {'User-Agent': 'Googlebot',}
    r = requests.get(url1,headers=headers)
    api1_data = json.loads(r.text)
    
    for i in range(n_items):
        itemid = api1_data['items'][i]['itemid']
        shopid = api1_data['items'][i]['shopid']
        
        url2 = f'https://shopee.tw/api/v2/item/get?itemid={itemid}&shopid={shopid}'
        r = requests.get(url2,headers=headers)
        api2_data = json.loads(r.text)
        output = api1_data['items'][i]['name'].ljust(50) +': ' + str(api2_data['item']['price']/100000)
        print(output)
        time.sleep(0.2)
    
    
    
shopeeAPI_Scraper(keyword = '溜冰鞋', n_items = 10)